## Event Price Comparer by Nicolas Levine, Owen Hovey, James Gagliardi, Ryan Weiss

Lo and behold, our SeatGeek v. Ticketmaster price comparer. In the is comparer you can make any combination of search terms in the two search boxes (if the search is speleed wrong, or there is no result, you will get an error). Once you have decided what you want to search, you can eaither search with SG or TM by using their repsctive buttons. Once you complete a successful search, you will be prompted with general info about tickets for the even you searched. If you see an even you want to research more, you may click on the "Buy Tickets" button to the right of the event and it will bring you to the respective webpage (use Chrome). This step was the hardest to implement, but is very satisfying now that we did. In order to do this, we needed to learn how to use lambda. With more time, we would work on adding a "search by date" option, and possibly integrating the websites directly into tkinter using some kind of webscraping.

In [2]:
import requests 
import json
import tkinter as tk
from tkinter import *
import time
import pprint
import webbrowser

sg_secret = "2d889596346beda669dcb39cb949d410ab12b68347d68d631697bef092891a9b"
sg_id = "MzMwNjgxMjh8MTY4MTQ5Njg4NC42NjYyNzc0"

tm_apikey = "Ez3L1QyVXpDzd4qTzySrYr9UcDQxuxxX"

sg_label_list = []
tm_label_list = []

def clear_lists():
    global sg_label_list
    global tm_label_list
    
    for label in sg_label_list:
        label.destroy()
        
    sg_label_list.clear()
    
    for label in tm_label_list:
        label.destroy()
        
    tm_label_list.clear()

def sg_search_performers():
    
    global sg_label_list
    clear_lists()
    
    query = query_entry.get()
    location = search_loc_entry.get()

    url = 'https://api.seatgeek.com/2/events'
    params = {'q': query, 'venue.city': location, 'client_id': sg_id}

    response = requests.get(url, params=params)
    events = response.json()
    options = events['events']
    
    if len(options) == 0:
        error = "Error: Unrecognized Query"
        error_label = Label(window, text=error, background="light blue", fg="black")
        error_label.grid(row=0, column=4, padx=3, pady=2, rowspan=3)

        sg_label_list.extend([error_label])
    
    for event in range(0, len(options)):
        
        name = options[event]['title']
        location = options[event]['venue']['display_location']
        venue = options[event]['venue']['name_v2']
        date = options[event]['datetime_local']
        
        if options[event]['stats']['lowest_price'] == None:
            min_price = "N/A"
            max_price = "N/A"
        else:
            min_price = f"${options[event]['stats']['lowest_price']}"
            max_price = f"${options[event]['stats']['highest_price']}" 
            
        buy_tickets_button = Button(window, text="Buy Tickets", background="orange red")
        
        if 'url' in options[event]:
            #https://realpython.com/python-lambda/#python-lambda-and-regular-functions
            buy_tickets_button.config(state=NORMAL, command=lambda url=options[event]['url']: webbrowser.open(url))
        buy_tickets_button.grid(row=event+5, column=6)
        
        name_label = Label(window, text=name, wraplength = 200, background="light blue", fg="black")
        name_label.grid(row=event+5, column=0)
        
        day_label = Label(window, text=date, background="light blue", fg="black")
        day_label.grid(row=event+5, column=1)
        
        event_loc_label = Label(window, text=location, wraplength= 150, background="light blue", fg="black")
        event_loc_label.grid(row=event+5, column=2)

        event_ven_label = Label(window, text=venue, wraplength= 150, background="light blue", fg="black")
        event_ven_label.grid(row=event+5, column=3)
        
        min_price_label = Label(window, text=min_price, background="light blue", fg="black")
        min_price_label.grid(row=event+5, column=4)
        
        max_price_label = Label(window, text=max_price, background="light blue", fg="black")
        max_price_label.grid(row=event+5, column=5)
        
        sg_label_list.extend([name_label, day_label, event_loc_label, event_ven_label, min_price_label, max_price_label, buy_tickets_button])
        
    sg_search_button.config(command = sg_new_search)
        
def sg_new_search():
    
    sg_search_performers()


def tm_search_performers():
    
    global tm_label_list
    clear_lists()
    
    query = query_entry.get()
    location = search_loc_entry.get()
    
    url = 'https://app.ticketmaster.com/discovery/v2/events.json'
    params = {'keyword': query, 'apikey': tm_apikey, 'city': location}
    
    try: 
        response = requests.get(url, params=params)
        events = response.json()['_embedded']['events']
        sorted_events = sorted(events, key=lambda event: event['dates']['start']['localDate'])

        for event in range(0, len(sorted_events)):
            name = sorted_events[event]['name']

            if 'priceRanges' in sorted_events[event]:
                min_price = sorted_events[event]['priceRanges'][0]['min']
                max_price = sorted_events[event]['priceRanges'][0]['max']

            else:
                min_price = 'N/A'
                max_price = 'N/A'

            buy_tickets_button = Button(window, text="Buy Tickets", background="blue", fg='white')
            if 'url' in sorted_events[event]:
                buy_tickets_button.config(state=NORMAL, command=lambda url=sorted_events[event]['url']: webbrowser.open(url))
            buy_tickets_button.grid(row=event+5, column=6)    


            location = sorted_events[event]['_embedded']['venues'][0]['city']['name'] 
            venue = sorted_events[event]['_embedded']['venues'][0]['name'] #issue with location, seems to only work if a city is used, not a state for example
            date = sorted_events[event]['dates']['start']['localDate'] 

            name_label = Label(window, text=name, background="light blue", fg="black")
            name_label.grid(row=event+5, column=0)

            day_label = Label(window, text=date, background="light blue", fg="black")
            day_label.grid(row=event+5, column=1)

            event_loc_label = Label(window, text=location, wraplength= 150, background="light blue", fg="black")
            event_loc_label.grid(row=event+5, column=2)

            event_ven_label = Label(window, text=venue, wraplength=400, background="light blue", fg="black")
            event_ven_label.grid(row=event+5, column=3)

            min_price_label = Label(window, text=min_price, background="light blue", fg="black")
            min_price_label.grid(row=event+5, column=4)

            max_price_label = Label(window, text=max_price, background="light blue", fg="black")
            max_price_label.grid(row=event+5, column=5)

            tm_label_list.extend([name_label, day_label, event_loc_label, event_ven_label, min_price_label, max_price_label, buy_tickets_button])

        tm_search_button.config(command = tm_new_search)
        
    except:
        error = "Error: Unrecognized Query"
        error_label = Label(window, text=error, background="light blue", fg="black")
        error_label.grid(row=0, column=4, padx=3, pady=2, rowspan=3)

        tm_label_list.extend([error_label])
        
def tm_new_search():
    
    tm_search_performers()
    
window = tk.Tk()
window.title("SeatGeek/Ticketmaster Performer Search")
window.configure(background="light blue")

label_titles = ["Event", "Date", "Location", "Venue", "Min Price", "Max Price"]

for i in label_titles:
    index = label_titles.index(i)
    col_label = Label(window, text=i, background="light blue", fg="black", font='Helvetica 10 bold')
    col_label.grid(row=4, column=index, pady=10)

query_label = Label (text="General Search (e.g. concert, boston celtics, msg):", background="light blue", fg="black", font='Helvetica 10 bold')
query_label.grid(row=1, column=0)
query_entry = Entry()
query_entry.grid(row=1, column=1, padx=5)

search_loc_label = Label (text="City (e.g. los angeles):", background="light blue", fg="black", font='Helvetica 10 bold')
search_loc_label.grid(row=2, column=0)
search_loc_entry = Entry()
search_loc_entry.grid(row=2, column=1, padx=5)


sg_search_button = Button(text="Search Seatgeek", command=sg_search_performers, background="orange red", fg="black")
sg_search_button.grid(row=0, column=2, padx=3, pady=2, rowspan=3)

tm_search_button = Button(text="Search Ticketmaster", command=tm_search_performers, background="blue", fg="white")
tm_search_button.grid(row=0, column=3, padx=5, pady=2, rowspan=3)

window.mainloop()